# **Llama 3.1 Fine Tuning**

## Llama 8B Fine-Tuning

Llama gibi büyük LLM modelleri oldukça büyük veri setleri eğitilir. Fine-Tuning ise bu büyük LLM modellerini kendi veri setimize göre tekrardan eğitmeyi amaçlamaktadır. Bu notebookta sizlere bir fine-tune edilme işlemi nasıl gerçekleştirilir bunu anlatacağız.

Bir modeli fine-tune etme işleminden önce veriye uygun model seçimi ve gerekli kütüphanelerin indirilmesi gereklidir. Bu kütüphanelerden bir tanesi ise unsloth kütüphanesidir.

Unsloth, büyük dil modellerini (LLM'leri) ince ayarlama (fine-tuning) ve eğitme süreçlerini hızlandıran bir Python paketidir. Hugging Face Transformers kütüphanesi üzerine kurulu olan Unsloth, sınırlı GPU kaynaklarıyla çalışan kullanıcılar için oldukça yararlıdır. Özellikle zaman ve bellek kullanımı açısından verimlilik sağlayarak, model eğitim süreci önemli ölçüde kısatlmaktadır.

Bu notebookta unsloth'un 0.0.28.post2 paketi ve sürüm numarasını kullanacağız. Bundan dolayı bunu pip install ile projemize dahil edelim.

In [1]:
!pip install unsloth "xformers==0.0.28.post2"

Unsloth'u projeye dahil ettik.

In [2]:
# kütüphanelerin import edilmesi
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # modelin işleyebileceği maksimum dizi uzunluğunu belirtmektedir.
dtype = None # veri tipi none olarak bırakılmış. Bu şekilde model, varsayılan veri tipini kullanacaktır.
load_in_4bit = True # modelin 4-bit hassasiyetinde yüklenmesini sağlar. Bu, modelin bellekte daha az yer kaplamasına ve daha hızlı çalışmasına olanak sağlar.

fourbit_models = [ #4-bit hassasiyetli bir model adı listesi oluşturulmuştur.
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # metanın Llama 3.1 8B parametreleri modeli yükleniyor
    max_seq_length = max_seq_length, # modelin bellekte daha verimli ve daha hızlı çalışmasını sağlamaktadır.
    dtype = dtype, # modelin bellekte daha verimli ve daha hızlı çalışmasını sağlamaktadır.
    load_in_4bit = load_in_4bit # modelin bellekte daha verimli ve daha hızlı çalışmasını sağlamaktadır.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Böylelikle model yüklenmiş olmaktadır. Şimdi üst parametreleri ayarlayarak modeli fine-tune etme işlemine devam edelim.

PEFT, model üzerinde daha az parametreyle ince ayar yapmayı sağlamaktadır. Bu, büyük dil modellerini çok fazla belleğe ihtiyaç duymadan optimize etmenin bir yolu olarak karşımıza çıkmaktadır.

In [3]:
model = FastLanguageModel.get_peft_model( # PEFT olarak ayarlandı
    model,
    r = 16, # LoRA projeksiyonunun boyutunu belirler. r=16 ile modelde yalnızca sınırlı sayıda parametre değiştirilecek şekilde adapte edilecektir.
    # ince ayar yapılacak modelin modüllerini tanımalamaktadır.
    # Aşağıdaki listede ismi geçen katmanlar seçilmektedir.
    # Bu katmanların seçilmesi özellikle büyük modellerde büyük bir avantaj sağlamaktadır
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16, # ile modelde değiştirilen parametrelerin etkisi, varsayılan değerine göre daha fazla arttırılmaktadır.
    lora_dropout = 0, #LoRA'DA dropout uygulaması için oranı belirlemektedir. Burada girilen 0 değeri, bu katmanda dropout uygulanmayacağını belirtmektedir.
    bias = "none", # ince ayar yapılırken bias değerlerini nasıl işleyeceğini ifade etmektedir burada none değeri verilmesi bias değeri verilmeyeceği anlamına gelmektedir.
    use_gradient_checkpointing = "unsloth", # model eğitilirken belirli adımları kaydedip tekrar eğitmeyi amaçlamaktadır. Bu şekilde efektif bir bellek yönetimi söz konusu olabilmektedir.
    random_state = 3407, # rastgelelik için başlangıç durumunu belirler, böylece sonuçların yeniden üretilebilir olmasını sağlamaktadır.
    use_rslora = False, # Random sparse Low-Rank özelliği devre dışı bırakıldı.
    loftq_config = None, # Low-OverHead Fine Tuning Quantization yapılandırılması tanımlandı.
)


Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Bu ayarlarla, modelin bellekte daha az yer kaplayarak etkili bir şekilde ince ayar yapılması sağlanır. Özellikle büyük dil modellerinde, bu yaklaşım performansı arttırırken kayank kullanımı oldukça minimize etmeye olanak sağlamaktadır.

In [4]:
# modelin vereceği cevapların formatlanacağı şekli verilmektedir. Bu şekilde prompt, Instruction, Input, Output şeklinde model cevaplamalar gerçekleştirecektir.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# modelin kelime parçacıkları arasındaki sınırlayıcı olan "end of sequence" tokenini alır ve her metin parçasının sonuna eklemektedir.
EOS_TOKEN = tokenizer.eos_token

# modelin çıkaracağı sonucu bir alpaca prompt formatında oluşturmaktadır.
def formatting_prompts_func(examples):
  instructions = examples["instruction"]
  inputs = examples["input"]
  outputs = examples["output"]
  texts = []
  for instruction, input, output in zip(instructions,inputs, outputs): # bütün değerler burada birleştirilir.
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text) #texts listesine text sözlüğü verilmektedir.
  return {"text" : texts, }
pass

# veri kümesini yükleme ve formatlama
from datasets import load_dataset
dataset = load_dataset("TFLai/Turkish-Alpaca", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True) # batched true ile veri toplu halde işlenmektedir.

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    # eğitim parametreleri
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),

)

max_steps is given, it will override any value given in num_train_epochs


###Model ve Tokenizer:

**model ve tokenizer:** Eğitilecek model ve tokenlaştırıcı. Daha önce tanımlanan model ve tokenizer burada kullanılıyor.

### Eğitim Verisi:

* ***train_dataset:*** dataset değişkeni ile sağlanmış eğitim veri kümesi.

* ***dataset_text_field:*** Modelin girdi olarak kullanacağı veri alanı. Bu durumda "text" alanı kullanılıyor.

* ***max_seq_length:*** Modelin işlem yapacağı maksimum dizi uzunluğu olarak ayarlanmış.

* ***dataset_num_proc:*** Veri kümesinin işlenmesi için kaç iş parçacığı kullanılacağını belirler. Bu örnekte 2 iş parçacığı kullanılıyor.

### Eğitim Parametreleri (TrainingArguments):

* ***per_device_train_batch_size:*** Her bir cihazda (örneğin GPU) kullanılacak batch boyutunu belirtir. 2 olarak ayarlanmış.

* ***gradient_accumulation_steps:*** 4 adım boyunca gradyan biriktirme yaparak her bir güncelleme arasında daha fazla veri kullanmayı sağlar.

* ***warmup_steps:*** Eğitim sürecinde 5 adımlık bir ısınma süresi ayarlanmış.
max_steps: Eğitim sürecinde yapılacak toplam adım sayısını 60 olarak ayarlar.

* ***learning_rate:*** Öğrenme oranı 2e-4 (0.0002) olarak ayarlanmış.
fp16 ve bf16: fp16 veya bf16 hassasiyet kullanımı. Donanım destekliyorsa bfloat16 (bf16) kullanılacak, aksi takdirde fp16.

* ***logging_steps:*** Her bir eğitim adımında çıktı bilgisi almak için 1 olarak ayarlanmış.

* ***optim:*** "adamw_8bit", bellekte verimli olan 8-bit AdamW optimizasyon algoritmasını kullanıyor.
weight_decay: 0.01 ağırlık azaltma oranı, aşırı öğrenmeyi önlemek için kullanılır.

* ***lr_scheduler_type:*** Öğrenme oranı düzenleyicisi olarak linear (doğrusal) tipi seçilmiş.

* ***seed:*** Rastgelelik için sabit bir başlangıç değeri (3407) belirler.

* ***output_dir:*** Eğitim çıktılarının kaydedileceği dizin (outputs).

* ***report_to:*** none olarak ayarlanmış, yani eğitim sürecinde herhangi bir izleme aracı (örneğin TensorBoard) kullanılmayacak.

In [6]:
# @title Meraklısına: Mevcut kullanılan T4 ekran başlangıçta ayırdığı bellek miktarı
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [7]:
trainer_stats = trainer.train() # modeli eğitelim

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,914 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.598100
2,2.659600
3,2.711900
4,2.543700
5,1.988300
6,2.006300
7,1.869300
8,1.663500
9,1.333200
10,1.439100


In [8]:
# @title Meraklısına: Ne kadar bellek kullanıldığı, ne kadar süre harcandığını belirtir.
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

353.8546 seconds used for training.
5.9 minutes used for training.
Peak reserved memory = 7.826 GB.
Peak reserved memory for training = 1.842 GB.
Peak reserved memory % of max memory = 53.065 %.
Peak reserved memory for training % of max memory = 12.49 %.


In [9]:
FastLanguageModel.for_inference(model)
inputs = tokenizer (
    [
        alpaca_prompt.format(
            "Bir sayı kümesi verildiğinde, maksimum değeri bulun.", # instruction
            "Küme: {10, 3, 25, 6, 16}", #input
            "", #output
        )
    ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBir sayı kümesi verildiğinde, maksimum değeri bulun.\n\n### Input:\nKüme: {10, 3, 25, 6, 16}\n\n### Response:\n25<|end_of_text|>']

In [10]:
FastLanguageModel.for_inference(model) # çıktıyı biraz daha düzenleyelim
inputs = tokenizer (
    [
        alpaca_prompt.format(
            "Bir sayı kümesi verildiğinde, maksimum değeri bulun.", # instruction
            "Küme: {10, 3, 25, 6, 16}", #input
            "", #output
        )
    ], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Bir sayı kümesi verildiğinde, maksimum değeri bulun.

### Input:
Küme: {10, 3, 25, 6, 16}

### Response:
25<|end_of_text|>


In [11]:
model.save_pretrained("lora_model") # modeli kaydedelim
tokenizer.save_pretrained("lora_model") # tokenizer yapılandırmasını da kaydedelim

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [13]:
if False:  # Model daha önceden varsa burada yüklenebilir ancak önceden yüklemediğimizi varsaydığımız için false ile başlattık.
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 2 kat daha hızlı yüklenir

# alpaca_prompt = You MUST copy from above!
inputs = tokenizer(
    [
        alpaca_prompt.format(  # alpaca formatını verelim
            "Doğal Dil İşlemede kelime gömme kullanımını açıklayın.",  # instruction
            "",  # input
            "",  # output - kendisi cevaplasın
        )
    ], return_tensors="pt"
).to("cuda")  # Nvidia CUDA çekirdeklerine taşıyarak daha hızlı işlem gerçekleştirsin

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)  # En fazla 128 yeni token üret


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Doğal Dil İşlemede kelime gömme kullanımını açıklayın.

### Input:


### Response:
Doğal Dil İşlemi'nde kelime gömme, kelimelerin bir araya getirilmesiyle oluşturulan yeni bir kelimeye atıfta bulunulduğunda ortaya çıkar. Kelime gömme, kelime yapısı, sözcük anlamları ve sözdizimi hakkında bilgi sağlamak için kullanılır. Örneğin, "kabuk" kelimesi, "deri" kelimesi ile bir araya getirildiğinde, "deri kabuğu" kelimesi oluşturulur.<|end_of_text|>


In [14]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

In [15]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

In [16]:
# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

In [17]:
# gradio arayüzü
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

In [18]:
!pip install gradio

In [19]:
from unsloth import FastLanguageModel

# modeli yükleyelim
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # lora modelimiz
    max_seq_length= 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model) # hızlı çıkarım için optimize edilmesi

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [20]:
# kütüphaneyi import etme
import gradio as gr

# gradio arayüzü oluşturma

def generate_response(instruction):
  # girdilerin hazırlanması
  inputs = tokenizer(
      [alpaca_prompt.format(instruction, "","")],
      return_tensors = "pt"
  ).to("cuda")

  # yanıt üretme
  generated_output = model.generate(**inputs, max_new_tokens = 256)

  decoded_output = tokenizer.batch_decode(generated_output, skip_special_tokens = True)

  # çıktıyı çözümleme
  return decoded_output[0].split("### Response:")[-1].strip() if decoded_output else "Cevap bulunamadı"

# gradio arayüzü
iface = gr.Interface(
    fn = generate_response, # fonksiyon
    inputs = "text", # girdi tipi
    outputs = "text", # çıktı tipi
    title = "Llama Fine-Tuning Modeli",
    description = "Fine tune edilmiş modeli test etmek için bir soru giriniz..."
)

# arayüzü çalıştıralım
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4300cc13de317f07a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
